In [20]:
from prompts import PROMPT_ASSUNTO, PROMPT_DIFICULDADE, PROMPT_SUBASSUNTO, REGRAS_SAIDA
import json
import os
import pandas as pd
from src.codes.questao import Questao

Como deve ser chamado cada um dos prompts

```python
assunto = PROMPT_ASSUNTO.format(
	enunciado="Teste", 
	regras_da_saida=REGRAS_SAIDA.format(
		regra="Fundamentos da Matemática, Fundamentos de Computação ou Algoritmos e Estruturas de Dados"
	)
)

dificuldade = PROMPT_DIFICULDADE.format(
	enunciado="Teste", 
	regras_da_saida=REGRAS_SAIDA.format(
		regra="Fácil, Médio ou Difícil"
	)
)
subassunto = PROMPT_SUBASSUNTO.format(
	enunciado="Teste", 
	assunto_principal="Algoritmos e Estruturas de Dados",
	conteudo_estruturado_subassuntos = subassuntos('Algoritmos e Estruturas de Dados'),
	regras_da_saida=REGRAS_SAIDA.format(
		regra="<Grupo>, <Subassunto>, <Nível>"
	)
)
```

In [2]:
with open('src/data/conteudo.json', 'r') as file:
	conteudo = json.load(file)

In [14]:
def subassuntos(assunto: str) -> str:
	"""
	Retorna os subassuntos disponíveis para o assunto especificado.

	Args:
		assunto (str): O assunto para o qual se deseja obter os subassuntos.

	Returns:
		str: Uma string formatada contendo os subassuntos e seus tópicos correspondentes.
	"""
	if assunto == 'Fundamentos de Matemática':
		indice = 0
	elif assunto == 'Fundamentos de Computação':
		indice = 1
	elif assunto == 'Algoritmos e Estruturas de Dados':
		indice = 2
	
	subassunto = ""
	for i,topico in enumerate(conteudo['conteudosEstruturados'][indice]['subsecoes'], start=1):
		subassunto += f"{i}. {topico['titulo']}\n"
		for subtopico in topico['topicos']:
			subassunto += f"\t[{subtopico['nivel']}] {subtopico['descricao']}\n"

	return subassunto

In [21]:
link_questoes = pd.read_csv('questoes.csv')